# Chemical potential in 3D lattice

## Introduction

This notebook gives a simple illustration to the chemical potential if Fermions in 3D, following the Fermi-Dirac statistics and density of states.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive as jupyter_interactive
from ipywidgets import interact

%matplotlib notebook

In [41]:
def interactive_chem(kt):
    '''
    :param k: wave number
    '''
    kBT = 0.001 # Boltzmann constant measured interms of chemical potential
    E_min = 0.0001
    E_max = 3.00
    factor = 1.0
    N = 200

    # Set up the variable space for 1D plot
    En_mu = np.linspace(E_min, E_max, N) #Energy measured in units of chemical potential
    
    f = 1.0/(np.exp( (factor*En_mu-1)/kt )+1) # Occupation number according to F-D statistics
    df = np.gradient(f) # Take the first derivative of the F-D distribution.
   
    
    c_f = np.sqrt(En_mu)/(np.exp( (factor*En_mu-1)/kt )+1) # Occupation number according to F-D statistics
    c_df = np.gradient(f) # Take the first derivative of the F-D distribution.
    
    plt.subplot(121)
    plt.plot(En_mu, f)
    plt.plot(En_mu, c_f)
    
    plt.subplot(122)
    plt.plot(En_mu, -df)
    plt.plot(En_mu, -c_df)


In [42]:
interact(interactive_chem,kt=(0.0001,0.10,0.001))

interactive(children=(FloatSlider(value=0.049100000000000005, description='kt', max=0.1, min=0.0001, step=0.00…

<function __main__.interactive_chem(kt)>